# River Flow Forecasting with DA-RNN

In [ ]:
from google.colab import auth
from datetime import datetime
auth.authenticate_user()
#!gcloud source repos clone github_aistream-peelout_flow-forecast --project=gmap-997
!git clone https://github.com/AIStream-Peelout/flow-forecast.git 

Cloning into 'flow-forecast'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 11970 (delta 193), reused 173 (delta 100), pack-reused 11681
Receiving objects: 100% (11970/11970), 3.80 MiB | 14.10 MiB/s, done.
Resolving deltas: 100% (8636/8636), done.


In [ ]:
import os
os.chdir('flow-forecast')
!pip install -r  requirements.txt
!python setup.py develop
!mkdir data

In [ ]:
!wandb login
import os
os.environ['MODEL_BUCKET'] = "predict_cfs"
os.environ["ENVIRONMENT_GCP"] = "Colab"
os.environ["GCP_PROJECT"] = "gmap-997"

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


 ### Downloading data and defining configuration


In [ ]:
# Full Flow-DB Dataset
#!gsutil -m cp -r gs://predict_cfs/day_addition .
#import pandas as pd
#for file_path in os.listdir('joined'):
  #the_file = pd.read_csv("joined/" + file_path)
  #the_file['month'] = pd.to_datetime(the_file['hour_updated']).map(lambda x: x.month)
  #the_file['month'] = the_file['month'].fillna(value=pd.to_datetime(the_file['datetime']).map(lambda x: x.month))
  #the_file['weekday'] = pd.to_datetime(the_file['hour_updated']).map(lambda x: x.weekday())
  #the_file.to_csv('add_thing2/'+file_path)

In [ ]:
import pandas as pd
!gsutil cp gs://aistream-datasets/flowdb/09405500AZC_flow.csv .
!gsutil cp gs://predict_cfs/virgin_final1.csv virgin_final.csv 
!gsutil cp -r gs://predict_cfs/joined_final_3 .
df = pd.read_csv("09405500AZC_flow.csv")
df = df.dropna(subset=["hour_updated", "cfs", "p01m"])
df.to_csv("/09405500AZC_flow.csv")

Copying gs://aistream-datasets/flowdb/09405500AZC_flow.csv...
- [1 files][ 19.6 MiB/ 19.6 MiB]                                                
Operation completed over 1 objects/19.6 MiB.                                     
Copying gs://predict_cfs/virgin_final1.csv...
/ [1 files][  9.2 MiB/  9.2 MiB]                                                
Operation completed over 1 objects/9.2 MiB.                                      
Copying gs://predict_cfs/joined_final_3/0101000040B_flow.csv...
Copying gs://predict_cfs/joined_final_3/0101007040B_flow.csv...
Copying gs://predict_cfs/joined_final_3/01010500FVE_flow.csv...
Copying gs://predict_cfs/joined_final_3/0101100040B_flow.csv...
\ [4 files][ 98.5 MiB/ 98.5 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4,7,8,9,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
import json
import os
import subprocess as subp
from flood_forecast.trainer import train_function
import traceback
from flood_forecast.long_train import split_on_letter

#!gsutil cp -r add_thing2 gs://predict_cfs/day_addition
def loop_through(data_dir:str, interrmittent_gcs=False, use_transfer=True, start_index=0, end_index=25): 
  """
  Function that makes and executes a set of config files
  This is since we have over 9k files.
  """
  if not os.path.exists("model_save"):
    os.mkdir("model_save")
  sorted_dir_list = sorted(os.listdir(data_dir))
  #total = len(sorted_dir_list)
  previous_gage = ""
  prev_config = {"pretrained_rivers":[]}
  for i in range(start_index, end_index):
      file_name = sorted_dir_list[i]
      station_id_gage = file_name.split("_flow.csv")[0]
      res = split_on_letter(station_id_gage)
      gage_id = res[0]
      station_id = res[1]
      file_path_name = os.path.join(data_dir, file_name)
      correct_file = None
      if use_transfer and len(os.listdir("model_save")) >0:
        print("Utilizing transfer learning")
        weight_files = filter(lambda x: x.endswith(".pth"), os.listdir("model_save"))
        paths = [] 
        for weight_file in weight_files:
             paths.append(os.path.join("model_save", weight_file)) 
        correct_file = max(paths, key = os.path.getctime)
        prev_config["pretrained_rivers"].append(previous_gage)
      sweep_full = wandb.sweep(wandb_sweep_config_full, project="github_aistream-peelout_flow-forecast")
      #sweep_id = "21i08e3p"
      wandb.agent(sweep_full, lambda:train_function("PyTorch", make_config_file(file_path_name, gage_id, station_id, correct_file, prev_config["pretrained_rivers"])))
      full_config = make_config_file(file_path_name, gage_id, station_id, correct_file, prev_config["pretrained_rivers"])
      try: 
        #model = train_function("PyTorch", config)
        print("temp disable")
      except Exception as e:
        print("An exception occured for: " + file_name_json)
        traceback.print_exc()
        print(e)
      previous_gage = gage_id

def make_config_file(flow_file_path, gage_id, station_id, weight_path=None, pretrained=[]):
  run = wandb.init(project="github_aistream-peelout_flow-forecast")
  wandb_config = run.config
  the_wandb_c = run.config
  print(wandb_config)
  the_config4 = {"model_name": "DARNN",
        "model_type": "PyTorch",
        "model_params": {
        "n_time_series":8,
        "forecast_history":wandb_config["forecast_history"],
        "hidden_size_encoder":wandb_config["hidden_encoder"],
        "decoder_hidden_size": wandb_config["hidden_decoder"],
        "out_feats": 1,
        "dropout": wandb_config["dropout"],
        "gru_lstm": False},
       "dataset_params":{"class": "default",
       "num_workers":12,
       "pin_memory": True,
       "training_path": flow_file_path,
       "validation_path": flow_file_path,
       "test_path": flow_file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"] - 1,
       "forecast_length":wandb_config["forecast_length"],
       "scaler": "StandardScaler",
       "train_start":1000,
       "train_end": 50000,
       "valid_start":50001,
       "valid_end": 57000,
       "sort_column": "hour_updated",
       "test_start": 57000,
       "test_end":58000,
       "target_col": ["cfs"],
       "relevant_cols": ["cfs", "tmpf", "p01m", "dwpf"], 
       "interpolate":{
           "method":"back_forward_generic",
           "params":{
               "relevant_columns":["cfs", "tmpf", "p01m", "dwpf"]}
           },
      "feature_param":
          {
            "datetime_params":{
            "hour":"cyclical",
            "month": "cyclical"
            }
          }
       },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "SGD",
       #"criterion_params":{"baseline_method":"mean"},
    "optim_params":{
       "lr": the_wandb_c["lr"]
    },
       "epochs": 12,
       "batch_size":wandb_config["batch_size"]
    },
    "early_stopping":{
        "patience":3
    },
    "GCS": True,
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE", "MAPE"],
   "inference_params":
   {     
         "datetime_start":"2020-05-31",
          "hours_to_forecast":336, 
          "num_prediction_samples": 20,
          "test_csv_path":flow_file_path,
          "decoder_params":{
            "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1},
          "dataset_params":{
             "file_path": flow_file_path,
             "sort_column": "hour_updated",
             "scaling": "StandardScaler",
             "forecast_history": wandb_config["forecast_history"] - 1,
             "forecast_length":wandb_config["forecast_length"],
             "relevant_cols": ["cfs", "tmpf", "p01m", "dwpf"],
             "target_col": ["cfs"],
             "interpolate_param":{
                 "method":"back_forward_generic",
                 "params":{"relevant_columns":["cfs", "tmpf", "p01m", "dwpf"]}
                 }, 
            "feature_params":
        {
         "datetime_params":{
            "hour":"cyclical",
            "month": "cyclical"
         
     }
             }
          }
          } 
    }

      
  if weight_path:
    the_config4["weight_path"] = weight_path
  wandb.config.update(the_config4)
  print("config made")
  return the_config4
  
wandb_sweep_config_full = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "forecast_length":{
            "values":[1]}
            ,
        "batch_size": {
            "values": [200, 400, 600]
        },
        "lr":{
            "values":[0.001, 0.0001, .01]
        },
        "forecast_history":{
            "values":[14, 20, 24, 48]
        },
        "out_seq_length":{
            "values":[1]
        },
        "hidden_encoder":
        {
            "values":[32, 64, 128, 256]
        },
        "hidden_decoder":
        {
            # 32, 64, 128 
            "values":[32, 64, 128, 256]
        }, 
        "dropout":
        {
            "values": [0.1, 0.3, 0.5, 0.8]
        }
  
    }
}

In [ ]:
import wandb
df = pd.read_csv("09405500AZC_flow.csv")
df = df.dropna(subset=["hour_updated", "cfs", "p01m"])
df.to_csv("09405500AZC_flow.csv")
sweep_id = wandb.sweep(wandb_sweep_config_full, project="github_aistream-peelout_flow-forecast")
#sweep_id = "03l75ilz"
file_path = "09405500AZC_flow.csv"
wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file(file_path, "09405500", "AZC", None)))

Columns (2,4,7,8,9,18) have mixed types.Specify dtype option on import or set low_memory=False.


Create sweep with ID: jtfsv38q
Sweep URL: https://wandb.ai/igodfried/github_aistream-peelout_flow-forecast/sweeps/jtfsv38q


wandb: Agent Starting Run: 9pr5yodl with config:
wandb: 	batch_size: 200
wandb: 	dropout: 0.1
wandb: 	forecast_history: 14
wandb: 	forecast_length: 1
wandb: 	hidden_decoder: 32
wandb: 	hidden_encoder: 32
wandb: 	lr: 0.001
wandb: 	out_seq_length: 1
wandb: Currently logged in as: igodfried (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


{'batch_size': 200, 'dropout': 0.1, 'forecast_history': 14, 'forecast_length': 1, 'hidden_decoder': 32, 'hidden_encoder': 32, 'lr': 0.001, 'out_seq_length': 1}
config made
interpolate should be below
running feature fix code s
Relevant cols are
['cos_hour', 'sin_hour', 'cos_month', 'sin_month']
Now loading09405500AZC_flow.csv
scaling now
1
interpolate should be below
running feature fix code s
Relevant cols are
['cos_hour', 'sin_hour', 'cos_month', 'sin_month']
Now loading09405500AZC_flow.csv
scaling now
1
interpolate should be below
running feature fix code s
Relevant cols are
['cos_hour', 'sin_hour', 'cos_month', 'sin_month']
Now loading09405500AZC_flow.csv
scaling now
1
Using Wandb config:
{'batch_size': 200, 'dropout': 0.1, 'forecast_history': 14, 'forecast_length': 1, 'hidden_decoder': 32, 'hidden_encoder': 32, 'lr': 0.001, 'out_seq_length': 1, 'model_name': 'DARNN', 'model_type': 'PyTorch', 'model_params': {'n_time_series': 8, 'forecast_history': 14, 'hidden_size_encoder': 32, 'd


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

epoch,11
_runtime,1398
_timestamp,1613414574
_step,61
loss,0.14126
average_prediction_sharpe0,0.00218


epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
_runtime,▁▁▂▂▃▃▃▄▄▅▅▆▆▇▇▇████████████████████████
_timestamp,▁▁▂▂▃▃▃▄▄▅▅▆▆▇▇▇████████████████████████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▂▂▂▁▁▁▁▁▁
average_prediction_sharpe0,▁


wandb: Agent Starting Run: dtcpb0l0 with config:
wandb: 	batch_size: 200
wandb: 	dropout: 0.1
wandb: 	forecast_history: 14
wandb: 	forecast_length: 1
wandb: 	hidden_decoder: 256
wandb: 	hidden_encoder: 64
wandb: 	lr: 0.001
wandb: 	out_seq_length: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.10.19 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'batch_size': 200, 'dropout': 0.1, 'forecast_history': 14, 'forecast_length': 1, 'hidden_decoder': 256, 'hidden_encoder': 64, 'lr': 0.001, 'out_seq_length': 1}
config made
interpolate should be below
running feature fix code s
Relevant cols are
['cos_hour', 'sin_hour', 'cos_month', 'sin_month']
Now loading09405500AZC_flow.csv
scaling now
1
interpolate should be below
running feature fix code s
Relevant cols are
['cos_hour', 'sin_hour', 'cos_month', 'sin_month']
Now loading09405500AZC_flow.csv
scaling now
1
interpolate should be below
running feature fix code s
Relevant cols are
['cos_hour', 'sin_hour', 'cos_month', 'sin_month']
Now loading09405500AZC_flow.csv
scaling now
1
Using Wandb config:
{'batch_size': 200, 'dropout': 0.1, 'forecast_history': 14, 'forecast_length': 1, 'hidden_decoder': 256, 'hidden_encoder': 64, 'lr': 0.001, 'out_seq_length': 1, 'model_name': 'DARNN', 'model_type': 'PyTorch', 'model_params': {'n_time_series': 8, 'forecast_history': 14, 'hidden_size_encoder': 64, 

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
def make_config_file2(flow_file_path, gage_id, station_id, weight_path=None, pretrained=[]):
  run = wandb.init(project="github_aistream-peelout_flow-forecast")
  wandb_config = run.config
  the_wandb_c = run.config
  print(wandb_config)
  the_config4 = {"model_name": "DARNN",
        "model_type": "PyTorch",
        "model_params": {
        "n_time_series":8,
        "forecast_history":8,
        "hidden_size_encoder":32,
        "decoder_hidden_size": 64,
        "out_feats": 1,
        "dropout": 0.1,
        "gru_lstm": False},
       "dataset_params":{"class": "default",
       "num_workers":12,
       "pin_memory": True,
       "training_path": flow_file_path,
       "validation_path": flow_file_path,
       "test_path": flow_file_path,
       "batch_size":10,
       "forecast_history":8,
       "forecast_length":1,
       "scaler": "StandardScaler",
       "train_start":1000,
       "train_end": 50000,
       "valid_start":50001,
       "valid_end": 57000,
       "sort_column": "hour_updated",
       "test_start": 57000,
       "test_end":58000,
       "target_col": ["cfs"],
       "relevant_cols": ["cfs", "tmpf", "p01m", "dwpf"], 
       "interpolate":{
           "method":"back_forward_generic",
           "params":{
               "relevant_columns":["cfs", "tmpf", "p01m", "dwpf"]}
           },
      "feature_param":
          {
            "datetime_params":{
            "hour":"cyclical",
            "month": "cyclical"
            }
          }
       },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "SGD",
       #"criterion_params":{"baseline_method":"mean"},
    "optim_params":{
       "lr": the_wandb_c["lr"]
    },
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    },
    "early_stopping":{
        "patience":3
    },
    "GCS": True,
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE", "MAPE"],
   "inference_params":
   {     
         "datetime_start":"2020-05-31",
          "hours_to_forecast":336, 
          "num_prediction_samples": 20,
          "test_csv_path":flow_file_path,
          "decoder_params":{
            "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1},
          "dataset_params":{
             "file_path": flow_file_path,
             "sort_column": "hour_updated",
             "scaling": "StandardScaler",
             "forecast_history": 8,
             "forecast_length":1,
             "relevant_cols": ["cfs", "tmpf", "p01m", "dwpf"],
             "target_col": ["cfs"],
             "interpolate_param":{
                 "method":"back_forward_generic",
                 "params":{"relevant_columns":["cfs", "tmpf", "p01m", "dwpf"]}
                 }, 
            "feature_params":
        {
         "datetime_params":{
            "hour":"cyclical",
            "month": "cyclical"
         
     }
             }
          }
          } 
    }

      
  if weight_path:
    the_config4["weight_path"] = weight_path
  wandb.config.update(the_config4)
  print("config made")
  return the_config4

In [ ]:
train_function("PyTorch", make_config_file(file_path, "09406000", "AZC", None))